In [1]:
import pandas as pd

In [31]:
patients_df= pd.read_csv('Patient_Data.csv')
billing_df= pd.read_csv('Billing_Data.csv')

In [32]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [33]:
billing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [34]:
###2
### only billing relevent columns
billing_columns = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patients_billing_df = patients_df[billing_columns]

In [35]:
###3
### drop administrative columns
patients_df = patients_df.drop(columns=['ReceptionistID', 'CheckInTime'],errors='ignore')

In [36]:
###4
### total bill amount per department
department_revenue = (
    patients_billing_df
    .groupby('Department')['BillAmount'].sum().reset_index())

print(department_revenue)


    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [37]:
### 5
### remove duplicate patient records
patients_df = patients_df.drop_duplicates(subset='PatientID',keep='last')

In [38]:
### 6
###fill missing bill amount with mean
mean_bill = patients_billing_df['BillAmount'].mean()

patients_billing_df['BillAmount'] = patients_billing_df['BillAmount'].fillna(mean_bill)

In [39]:
### 7
### merge billing dataset with patient data
merged_df = pd.merge(
    patients_df,billing_df,on='PatientID',how='inner')

In [40]:
###8
### concatenate new patient data (row-wise)
new_patients_df = pd.DataFrame({
    'PatientID': [501, 502],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. Rao', 'Dr. Mehta'],
    'BillAmount': [15000, 18000]
})

final_df = pd.concat(
    [merged_df, new_patients_df],
    axis=0,
    ignore_index=True
)


In [43]:
### 9 
### concatenate new billing columns(colum-wise)
new_billing_columns = pd.DataFrame({'InsuranceCovered': [False] * len(final_df),'FinalAmount': final_df['BillAmount'] * 0.9},index=final_df.index)

final_df = pd.concat([final_df, new_billing_columns], axis=1)


In [45]:
department_revenue

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


In [47]:
do

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000
